In [1]:
import DANE
from dummyhandler import DummyHandler
import json

def pprint(obj):
    print(json.dumps(json.loads(str(obj)), indent=4))

# Job

In [2]:
job = DANE.Job(source_url='http://127.0.0.1/example', source_id='ITM123',
                      tasks=DANE.taskSequential(['DOWNLOAD', 'CV', 'ASR', 'INDEX', 'DELETE']),
                    metadata={'FOO': 'BAR'})

pprint(job)

{
    "source_url": "http://127.0.0.1/example",
    "source_id": "ITM123",
    "job_id": null,
    "tasks": {
        "taskSequential": [
            "DOWNLOAD",
            "CV",
            "ASR",
            "INDEX",
            "DELETE"
        ]
    },
    "metadata": {
        "FOO": "BAR"
    },
    "priority": 1,
    "response": {}
}


In [3]:
new_job = DANE.Job.from_json(job.to_json())
pprint(new_job)

{
    "source_url": "http://127.0.0.1/example",
    "source_id": "ITM123",
    "job_id": null,
    "tasks": {
        "taskSequential": [
            "DOWNLOAD",
            "CV",
            "ASR",
            "INDEX",
            "DELETE"
        ]
    },
    "metadata": {
        "FOO": "BAR"
    },
    "priority": 1,
    "response": {}
}


## API

In [4]:
# Dummy endpoint so we can 'simulate' behaviour of workflow
dummy = DummyHandler()
new_job.set_api(dummy)

print(new_job.tasks[0].api) # instance, won't be serialised with job

In [5]:
# register job & all tasks
new_job.register()
pprint(new_job)

{
    "source_url": "http://127.0.0.1/example",
    "source_id": "ITM123",
    "job_id": "aae88e57-4733-470b-820b-3b0c1bbd88df",
    "tasks": {
        "taskSequential": [
            {
                "Task": {
                    "task_key": "DOWNLOAD",
                    "task_id": "0",
                    "job_id": "aae88e57-4733-470b-820b-3b0c1bbd88df"
                }
            },
            {
                "Task": {
                    "task_key": "CV",
                    "task_id": "1",
                    "job_id": "aae88e57-4733-470b-820b-3b0c1bbd88df"
                }
            },
            {
                "Task": {
                    "task_key": "ASR",
                    "task_id": "2",
                    "job_id": "aae88e57-4733-470b-820b-3b0c1bbd88df"
                }
            },
            {
                "Task": {
                    "task_key": "INDEX",
                    "task_id": "3",
                    "job_id": "aae88e57-4733-470b-820b-3

In [6]:
# Explicitly run a specific task
# No guarantee that this task can run successfully if preceding tasks havent been run
new_job.tasks[0].run()

DummyEndpoint: Executed task DOWNLOAD for job: aae88e57-4733-470b-820b-3b0c1bbd88df


In [7]:
# run next task
new_job.tasks.run()

DummyEndpoint: Executed task CV for job: aae88e57-4733-470b-820b-3b0c1bbd88df


In [8]:
job.set_api(dummy)
job.register()

while not job.isDone():
    job.run()

DummyEndpoint: Executed task DOWNLOAD for job: f2eda91d-e7db-4181-947a-7bc11178b2ae
DummyEndpoint: Executed task CV for job: f2eda91d-e7db-4181-947a-7bc11178b2ae
DummyEndpoint: Executed task ASR for job: f2eda91d-e7db-4181-947a-7bc11178b2ae
DummyEndpoint: Executed task INDEX for job: f2eda91d-e7db-4181-947a-7bc11178b2ae
DummyEndpoint: Executed task DELETE for job: f2eda91d-e7db-4181-947a-7bc11178b2ae


# Nesting Task Containers

In [9]:
new_tasks = DANE.taskSequential(['DOWNLOAD', 
                            DANE.taskParallel(['CV', 'ASR']), 
                            DANE.taskParallel(['INDEX', 'DELETE'])])
pprint(new_tasks)

{
    "taskSequential": [
        "DOWNLOAD",
        {
            "taskParallel": [
                "CV",
                "ASR"
            ]
        },
        {
            "taskParallel": [
                "INDEX",
                "DELETE"
            ]
        }
    ]
}


In [10]:
newer_tasks = DANE.taskContainer.from_json(new_tasks.to_json())
pprint(newer_tasks)

{
    "taskSequential": [
        "DOWNLOAD",
        {
            "taskParallel": [
                "CV",
                "ASR"
            ]
        },
        {
            "taskParallel": [
                "INDEX",
                "DELETE"
            ]
        }
    ]
}


In [11]:
newer_job = DANE.Job(source_url='http://127.0.0.1/example2', source_id='ITM124', tasks=newer_tasks)

In [12]:
newer_job.set_api(dummy)
newer_job.register()

In [13]:
newest_job = DANE.Job.from_json(newer_job.to_json())
newest_job.set_api(dummy) # api isn't serialised, so need to set again

newest_job.tasks[0].run()
newest_job.tasks[1].run()

DummyEndpoint: Executed task DOWNLOAD for job: 21964668-bf6c-4fb5-89ea-5c8dc8e39a74
DummyEndpoint: Executed task CV for job: 21964668-bf6c-4fb5-89ea-5c8dc8e39a74
DummyEndpoint: Executed task ASR for job: 21964668-bf6c-4fb5-89ea-5c8dc8e39a74
